In [136]:
import pandas as pd
import numpy as np
from gurobipy import GRB, Model, quicksum

import utils.util as utils

In [137]:
cities_df = pd.read_csv('data/cities_mainline.csv')
# for i, row in cities_df.iterrows():
#     if row['city'] == ''
#     cities_df.at[i, 'has_switching_station'] = row['city']
cities_df['switching_stations'] = 0
cities_df.loc[cities_df['City'] == 'Opole', 'switching_stations'] = 1
cities_df.loc[cities_df['City'] == 'Warszawa', 'switching_stations'] = 2
cities_df.loc[cities_df['City'] == 'Kielce', 'switching_stations'] = 3
cities_df

,Area,City,x,y,TB/s,in_mainline,switching_stations
0,1,Warszawa,100,65,-,1,2
1,1,Skierniewice,92,62,197,1,0
2,1,Lódz,85,58,317,0,0
3,1,Konin,68,64,236,0,0
4,1,Sieradz,72,55,178,0,0
5,1,Piotrków,89,51,139,1,0
6,1,Radom,102,50,288,1,0
7,2,Kielce,98,42,-,1,3
8,2,Czestochowa,79,40,278,0,0
9,2,Bytom,76,31,163,0,0


### Parameters

In [138]:
S = cities_df[cities_df['switching_stations'] > 0]['City'].tolist()
C_not = cities_df[cities_df['in_mainline'] == 0]['City'].tolist()

M = {s: cities_df[cities_df['City'] == s]['switching_stations'].values[0]*400 for s in S}
R = {c_not: int(cities_df[cities_df['City'] == c_not]['TB/s'].values[0]) for c_not in C_not}
d = {(s, c_not): utils.euclidean((cities_df[cities_df['City'] == s]['x'].values[0], cities_df[cities_df['City'] == s]['y'].values[0]), (cities_df[cities_df['City'] == c_not]['x'].values[0], cities_df[cities_df['City'] == c_not]['y'].values[0])) for s in S for c_not in C_not}
P = 1000
L = 200

In [139]:
model = Model('Capacity optimization')

### Variables

In [140]:
f = model.addVars([(s, c_not) for s in S for c_not in C_not], vtype=GRB.CONTINUOUS, name='f')

### Objective function

In [141]:
model.setObjective(quicksum(P*d[(s, c_not)]*f[s, c_not] for s in S for c_not in C_not), GRB.MINIMIZE)

### Constraints

#### Copper cable constraint, degradation constraint, and non negativity constraint

In [142]:
for s in S:
    for c_not in C_not:
        model.addConstr(f[s, c_not] <= L, f'copper_cable_capacity_{s}_{c_not}')             # Copper cable capacity constraint
        model.addConstr(f[s, c_not] <= 2*np.pi*d[s, c_not], f'degradation_{s}_{c_not}')     # Degradation constraint
        model.addConstr(f[s, c_not] >= 0, f'non_negativity_{s}_{c_not}')                    # Non negativity constraint

#### City requirement constraint

In [143]:
for c_not in C_not:
    # print(R)
    # print(f[s, c_not] for c_not in C_not)
    # print(quicksum(f[s, c_not] for c_not in C_not).getValue())
    model.addConstr(quicksum(f[s, c_not] for s in S) >= R[c_not], f'city_requirement_{c_not}')

#### Maximum capacity constraint

In [144]:
for s in S:
    model.addConstr(quicksum(f[s, c_not] for c_not in C_not) <= M[s], f'maximum_capacity_{s}')

UFO constraint

In [145]:
for s in S:
    model.addConstr(f[s, 'Lódz'] <= 0.4 * R['Lódz'], f'UFO_{s}')

### Optimize

In [146]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 126 rows, 36 columns and 183 nonzeros
Model fingerprint: 0x1b6acff4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+04, 6e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+01, 1e+03]
Presolve removed 111 rows and 0 columns
Presolve time: 0.01s
Presolved: 15 rows, 36 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.7009370e+06   2.493404e+02   0.000000e+00      0s
      21    6.9349644e+07   0.000000e+00   0.000000e+00      0s

Solved in 21 iterations and 0.02 seconds (0.00 work units)
Optimal objective  6.934964446e+07


In [147]:
if model.status == GRB.OPTIMAL:
    model.printStats()
    print()
    
    # What will be the costs of connecting the remaining cities to the switching stations?
    objective_value = model.getObjective().getValue()
    print(f'Cost of connecting the remaining cities to the switching stations: {objective_value:.2f} EUR\n')
    
    # What capacity should be installed between Poznań and Wałbrzych and the variou switching stations?
    print('Poznań capacity:')
    for s in S:
        capacity_1 = f[s, 'Poznań'].X
        print(f'- Capacity between {s} and Poznań: {capacity_1:.2f}')
    print('\nWałbrzych capacity:')
    for s in S:
        capacity_2 = f[s, 'Wałbrzych'].X
        print(f'- Capacity between {s} and Wałbrzych: {capacity_2:.2f}')
        
    # How much of the total capacity at the switching station in Kielce is utilized?
    print('\nKielce capacity:')
    capacity_3 = sum(f['Kielce', c_not].X for c_not in C_not)
    print(f'- Total capacity at the switching station in Kielce: {M["Kielce"]:.2f}')
    print(f'- Utilized capacity at the switching station in Kielce: {capacity_3:.2f}')
    print(f'- Utilization: {(capacity_3/M["Kielce"])*100:.2f}%, Remaining capacity: {M["Kielce"]-capacity_3:.2f}')


Statistics for model Capacity optimization:
  Linear constraint matrix    : 126 Constrs, 36 Vars, 183 NZs
  Matrix coefficient range    : [ 1, 1 ]
  Objective coefficient range : [ 10816.7, 61684.7 ]
  Variable bound range        : [ 0, 0 ]
  RHS coefficient range       : [ 67.963, 1200 ]

Cost of connecting the remaining cities to the switching stations: 69349644.46 EUR

Poznań capacity:
- Capacity between Warszawa and Poznań: 200.00
- Capacity between Kielce and Poznań: 51.00
- Capacity between Opole and Poznań: 0.00

Wałbrzych capacity:
- Capacity between Warszawa and Wałbrzych: 0.00
- Capacity between Kielce and Wałbrzych: 0.00
- Capacity between Opole and Wałbrzych: 129.00

Kielce capacity:
- Total capacity at the switching station in Kielce: 1200.00
- Utilized capacity at the switching station in Kielce: 1131.00
- Utilization: 94.25%, Remaining capacity: 69.00


In [153]:
if model.status == GRB.OPTIMAL:
    for s in S:
        value = f[s, 'Konin'].X
        reduced_cost = f[s, 'Konin'].RC
        linear_objective_coefficient = f[s, 'Konin'].Obj
        objective_coefficient_sensitivity_information = f[s, 'Konin'].SAObjUp
        # slack = f[s, 'Konin'].slack
        # shadow_price = f[s, 'Konin'].Pi
        # right_hand_side_value = f[s, 'Konin'].RHS
        right_hand_side_sensistivity_information = f[s, 'Konin'].SAUBUp
    
        print(f'{s} -> Konin: {value:.2f} TB/s')
        print(f'Reduced cost: {reduced_cost:.2f}')
        print(f'Linear objective coefficient: {linear_objective_coefficient:.2f}')
        print(f'Objective coefficient sensitivity information: {objective_coefficient_sensitivity_information:.2f}')
        # print(f'Slack: {slack:.2f}')
        # print(f'Shadow price: {shadow_price:.2f}')
        # print(f'Right-hand side value: {right_hand_side_value:.2f}')
        print(f'Right-hand side sensitivity information: {right_hand_side_sensistivity_information:.2f}')
        print()
        
    # Assuming the city requirement constraint for Konin is available in the model
    for s in S:
        # Extracting sensitivity information for the demand constraint of Konin
        demand_sensitivity_upper = f[s, 'Konin'].SAUBUp
        demand_sensitivity_lower = f[s, 'Konin'].SALBUp
        
        print(f"Sensitivity range for Konin's demand: [{demand_sensitivity_lower}, {demand_sensitivity_upper}]")

Warszawa -> Konin: 200.00 TB/s
Reduced cost: 0.00
Linear objective coefficient: 32015.62
Objective coefficient sensitivity information: 36722.04
Right-hand side sensitivity information: inf

Kielce -> Konin: 36.00 TB/s
Reduced cost: 0.00
Linear objective coefficient: 37202.15
Objective coefficient sensitivity information: 55856.33
Right-hand side sensitivity information: inf

Opole -> Konin: 0.00 TB/s
Reduced cost: 18654.18
Linear objective coefficient: 27018.51
Objective coefficient sensitivity information: inf
Right-hand side sensitivity information: inf

Sensitivity range for Konin's demand: [200.0, inf]
Sensitivity range for Konin's demand: [36.0, inf]
Sensitivity range for Konin's demand: [36.0, inf]


#### Check cities

In [149]:
if model.status == GRB.OPTIMAL:
    
    results = {}
    results['City'] = C_not
    for s in S:
        results[f'{s}'] = [1 if f[s, c_not].X > 0 else 0 for c_not in C_not]
    for s in S:
        results[f'{s}_flow'] = [round(f[s, c_not].X, 2) for c_not in C_not]
    
    results['Flow_sum'] = [sum([f[s, c_not].X for s in S]) for c_not in C_not]
    results['Requirement'] = [R[c_not] for c_not in C_not]
    results['Difference'] = [R[c_not] - sum([f[s, c_not].X for s in S]) for c_not in C_not]
    
    results_df = pd.DataFrame(results)
    print(results_df)
        
    # connections_df = pd.DataFrame(columns=['City', 'Opole', 'Warszawa', 'Kielce'])

                City  Warszawa  Kielce  Opole  Warszawa_flow  Kielce_flow  \
0               Lódz         1       1      1         104.01       126.80   
1              Konin         1       1      0         200.00        36.00   
2            Sieradz         0       1      0           0.00       178.00   
3        Czestochowa         1       1      0         157.96       120.04   
4              Bytom         0       1      1           0.00       154.55   
5          Sosnowiec         1       1      0           3.49       139.51   
6           Katowice         1       1      1          20.31       153.39   
7   Wodzisław Śląski         0       1      0           0.00       119.00   
8             Poznań         1       1      0         200.00        51.00   
9       Zielona Góra         1       1      0         114.24         6.76   
10           Wrocław         0       1      1           0.00        45.95   
11         Wałbrzych         0       0      1           0.00         0.00   

#### Check switching stations

In [150]:
if model.status == GRB.OPTIMAL:
    
    results = {}
    results['Switching_station'] = S
    results['Maximum_capacity'] = [M[s] for s in S]
    results['Capacity_used'] = [sum([f[s, c_not].X for c_not in C_not]) for s in S]
    results['Remaining_capacity'] = [round(M[s] - sum([f[s, c_not].X for c_not in C_not]), 1) for s in S]
    
    results_df = pd.DataFrame(results)
    print(results_df)

  Switching_station  Maximum_capacity  Capacity_used  Remaining_capacity
0          Warszawa               800          800.0                 0.0
1            Kielce              1200         1131.0                69.0
2             Opole               400          400.0                 0.0
